In [2]:
# =====================================
# 🔹 1. Setup Environment
# =====================================
!pip install -q kaggle torch torchvision matplotlib tqdm

import os
import torch
import torch.nn as nn
import itertools
from torch.utils.data import DataLoader
from torchvision import transforms, datasets, utils
from tqdm import tqdm
import matplotlib.pyplot as plt

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("✅ Using device:", device)

# =====================================
# 🔹 2. Load Dataset from Kaggle
# =====================================
!mkdir -p ~/.kaggle
from google.colab import files
files.upload()  # Upload your kaggle.json here manually once

!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d almightyj/person-face-sketches -p /content/
!unzip -q /content/person-face-sketches.zip -d /content/dataset

# =====================================
# 🔹 3. Define Transforms and Dataloaders
# =====================================
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

class ImageDataset(torch.utils.data.Dataset):
    def __init__(self, root, transform=None, max_images=None):
        self.transform = transform
        self.files_A = sorted(os.listdir(os.path.join(root, 'photos')))
        self.files_B = sorted(os.listdir(os.path.join(root, 'sketches')))
        if max_images:
            self.files_A = self.files_A[:max_images]
            self.files_B = self.files_B[:max_images]
        self.root_A = os.path.join(root, 'photos')
        self.root_B = os.path.join(root, 'sketches')

    def __getitem__(self, index):
        from PIL import Image
        img_A = Image.open(os.path.join(self.root_A, self.files_A[index % len(self.files_A)])).convert('RGB')
        img_B = Image.open(os.path.join(self.root_B, self.files_B[index % len(self.files_B)])).convert('RGB')
        if self.transform:
            img_A = self.transform(img_A)
            img_B = self.transform(img_B)
        return {'A': img_A, 'B': img_B}

    def __len__(self):
        return min(len(self.files_A), len(self.files_B))

train_dataset = ImageDataset('/content/dataset/train', transform=transform, max_images=1000)
train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)

# =====================================
# 🔹 4. Define CycleGAN Models
# =====================================
class ResnetBlock(nn.Module):
    def __init__(self, dim):
        super().__init__()
        self.block = nn.Sequential(
            nn.Conv2d(dim, dim, 3, 1, 1),
            nn.InstanceNorm2d(dim),
            nn.ReLU(True),
            nn.Conv2d(dim, dim, 3, 1, 1),
            nn.InstanceNorm2d(dim)
        )
    def forward(self, x): return x + self.block(x)

class Generator(nn.Module):
    def __init__(self, input_nc, output_nc, n_blocks=6):
        super().__init__()
        model = [
            nn.Conv2d(input_nc, 64, 7, 1, 3),
            nn.InstanceNorm2d(64),
            nn.ReLU(True)
        ]
        in_channels = 64
        for out_channels in [128, 256]:
            model += [
                nn.Conv2d(in_channels, out_channels, 3, 2, 1),
                nn.InstanceNorm2d(out_channels),
                nn.ReLU(True)
            ]
            in_channels = out_channels
        for _ in range(n_blocks):
            model += [ResnetBlock(in_channels)]
        for out_channels in [128, 64]:
            model += [
                nn.ConvTranspose2d(in_channels, out_channels, 3, 2, 1, 1),
                nn.InstanceNorm2d(out_channels),
                nn.ReLU(True)
            ]
            in_channels = out_channels
        model += [nn.Conv2d(in_channels, output_nc, 7, 1, 3), nn.Tanh()]
        self.model = nn.Sequential(*model)
    def forward(self, x): return self.model(x)

class Discriminator(nn.Module):
    def __init__(self, input_nc):
        super().__init__()
        model = [
            nn.Conv2d(input_nc, 64, 4, 2, 1),
            nn.LeakyReLU(0.2, True)
        ]
        model += [
            nn.Conv2d(64, 128, 4, 2, 1),
            nn.InstanceNorm2d(128),
            nn.LeakyReLU(0.2, True),

            nn.Conv2d(128, 256, 4, 2, 1),
            nn.InstanceNorm2d(256),
            nn.LeakyReLU(0.2, True),

            nn.Conv2d(256, 512, 4, 1, 1),
            nn.InstanceNorm2d(512),
            nn.LeakyReLU(0.2, True),

            nn.Conv2d(512, 1, 4, 1, 1)
        ]
        self.model = nn.Sequential(*model)
    def forward(self, x): return self.model(x)

# Instantiate models
G_A2B = Generator(3, 3).to(device)
G_B2A = Generator(3, 3).to(device)
D_A = Discriminator(3).to(device)
D_B = Discriminator(3).to(device)

# =====================================
# 🔹 5. Define Losses and Optimizers
# =====================================
criterion_GAN = nn.MSELoss()
criterion_cycle = nn.L1Loss()

optimizer_G = torch.optim.Adam(itertools.chain(G_A2B.parameters(), G_B2A.parameters()), lr=0.0002, betas=(0.5, 0.999))
optimizer_D_A = torch.optim.Adam(D_A.parameters(), lr=0.0002, betas=(0.5, 0.999))
optimizer_D_B = torch.optim.Adam(D_B.parameters(), lr=0.0002, betas=(0.5, 0.999))

# =====================================
# 🔹 6. Training Loop (30 epochs, 1000 samples)
# =====================================
num_epochs = 30
save_path = '/content/drive/MyDrive/cyclegan_checkpoints'
os.makedirs(save_path, exist_ok=True)

for epoch in range(1, num_epochs + 1):
    G_A2B.train(); G_B2A.train()
    total_G_loss, total_D_loss = 0, 0

    for i, batch in enumerate(tqdm(train_loader, desc=f"Epoch {epoch}/{num_epochs}")):
        real_A = batch['A'].to(device)
        real_B = batch['B'].to(device)

        valid = torch.ones((real_A.size(0), 1, 30, 30), device=device)
        fake = torch.zeros((real_A.size(0), 1, 30, 30), device=device)

        # --- Train Generators ---
        optimizer_G.zero_grad()
        fake_B = G_A2B(real_A)
        fake_A = G_B2A(real_B)

        loss_GAN_A2B = criterion_GAN(D_B(fake_B), valid)
        loss_GAN_B2A = criterion_GAN(D_A(fake_A), valid)
        loss_cycle_A = criterion_cycle(G_B2A(fake_B), real_A)
        loss_cycle_B = criterion_cycle(G_A2B(fake_A), real_B)
        loss_G = loss_GAN_A2B + loss_GAN_B2A + 10.0 * (loss_cycle_A + loss_cycle_B)
        loss_G.backward()
        optimizer_G.step()

        # --- Train Discriminator A ---
        optimizer_D_A.zero_grad()
        loss_D_A = (criterion_GAN(D_A(real_A), valid) + criterion_GAN(D_A(fake_A.detach()), fake)) * 0.5
        loss_D_A.backward()
        optimizer_D_A.step()

        # --- Train Discriminator B ---
        optimizer_D_B.zero_grad()
        loss_D_B = (criterion_GAN(D_B(real_B), valid) + criterion_GAN(D_B(fake_B.detach()), fake)) * 0.5
        loss_D_B.backward()
        optimizer_D_B.step()

        total_G_loss += loss_G.item()
        total_D_loss += (loss_D_A.item() + loss_D_B.item())

    print(f"Epoch [{epoch}/{num_epochs}]  G_loss: {total_G_loss/len(train_loader):.4f}  D_loss: {total_D_loss/len(train_loader):.4f}")

    # Save checkpoints
    torch.save(G_A2B.state_dict(), f"{save_path}/G_A2B_epoch{epoch}.pth")
    torch.save(G_B2A.state_dict(), f"{save_path}/G_B2A_epoch{epoch}.pth")
    print(f"✅ Saved model checkpoints for epoch {epoch}")

# =====================================
# 🔹 7. Test Inference Example
# =====================================
from PIL import Image
test_path = '/content/dataset/test/sketches'
img = Image.open(os.path.join(test_path, os.listdir(test_path)[0])).convert('RGB')
img_tensor = transform(img).unsqueeze(0).to(device)
with torch.no_grad():
    fake_photo = G_B2A(img_tensor)
utils.save_image((fake_photo * 0.5 + 0.5), '/content/output_face.jpg')
print("✅ Output generated and saved at /content/output_face.jpg")


✅ Using device: cuda


Saving kaggle.json to kaggle.json
Dataset URL: https://www.kaggle.com/datasets/almightyj/person-face-sketches
License(s): CC0-1.0
person-face-sketches.zip: Skipping, found more recently modified local copy (use --force to force download)
replace /content/dataset/test/photos/10132.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
a


Epoch 1/30: 100%|██████████| 1000/1000 [04:49<00:00,  3.46it/s]


Epoch [1/30]  G_loss: 4.5230  D_loss: 0.3930
✅ Saved model checkpoints for epoch 1


Epoch 2/30: 100%|██████████| 1000/1000 [04:51<00:00,  3.43it/s]


...

Epoch [28/30]  G_loss: 3.2513  D_loss: 0.1522
✅ Saved model checkpoints for epoch 28


Epoch 29/30: 100%|██████████| 1000/1000 [04:51<00:00,  3.43it/s]


Epoch [29/30]  G_loss: 3.2307  D_loss: 0.1538
✅ Saved model checkpoints for epoch 29


Epoch 30/30: 100%|██████████| 1000/1000 [04:51<00:00,  3.43it/s]

Epoch [30/30]  G_loss: 3.2004  D_loss: 0.1502
✅ Saved model checkpoints for epoch 30
✅ Output generated and saved at /content/output_face.jpg


In [ ]:
# ============================================================
# 🔹 CycleGAN Flask UI — Modern frontend with side-by-side preview
# ============================================================

!pip install -q flask flask_cors pyngrok Pillow torchvision torch tqdm opencv-python

# --- Imports ---
import os, io, base64
from PIL import Image
from flask import Flask, request, jsonify, render_template_string
from flask_cors import CORS
from pyngrok import ngrok
import torch, torch.nn as nn
from torchvision import transforms
import numpy as np
import cv2

# --- Drive mount ---
if os.path.exists("/content/drive/MyDrive"):
    print("✅ Google Drive already mounted.")
else:
    from google.colab import drive
    drive.mount('/content/drive')
    print("✅ Mounted Google Drive.")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# ============================================================
# 🔹 Model Definitions
# ============================================================
class ResnetBlock(nn.Module):
    def __init__(self, dim):
        super().__init__()
        self.block = nn.Sequential(
            nn.Conv2d(dim, dim, 3, 1, 1),
            nn.InstanceNorm2d(dim),
            nn.ReLU(True),
            nn.Conv2d(dim, dim, 3, 1, 1),
            nn.InstanceNorm2d(dim)
        )
    def forward(self, x): return x + self.block(x)

class Generator(nn.Module):
    def __init__(self, input_nc, output_nc, n_blocks=6):
        super().__init__()
        model = [
            nn.Conv2d(input_nc, 64, 7, 1, 3),
            nn.InstanceNorm2d(64),
            nn.ReLU(True)
        ]
        in_channels = 64
        for out_channels in [128, 256]:
            model += [nn.Conv2d(in_channels, out_channels, 3, 2, 1),
                      nn.InstanceNorm2d(out_channels), nn.ReLU(True)]
            in_channels = out_channels
        for _ in range(n_blocks):
            model += [ResnetBlock(in_channels)]
        for out_channels in [128, 64]:
            model += [nn.ConvTranspose2d(in_channels, out_channels, 3, 2, 1, 1),
                      nn.InstanceNorm2d(out_channels), nn.ReLU(True)]
            in_channels = out_channels
        model += [nn.Conv2d(in_channels, output_nc, 7, 1, 3), nn.Tanh()]
        self.model = nn.Sequential(*model)
    def forward(self, x): return self.model(x)

# ============================================================
# 🔹 Load trained weights
# ============================================================
G_A2B_path = "/content/drive/MyDrive/cyclegan_checkpoints/G_A2B_epoch30.pth"
G_B2A_path = "/content/drive/MyDrive/cyclegan_checkpoints/G_B2A_epoch30.pth"

G_A2B = Generator(3,3).to(device)
G_B2A = Generator(3,3).to(device)

def load_model_safe(model, path):
    if os.path.exists(path):
        model.load_state_dict(torch.load(path, map_location=device))
        model.eval()
        print(f"✅ Loaded checkpoint: {path}")
    else:
        print(f"⚠️ Checkpoint not found: {path}")

load_model_safe(G_A2B, G_A2B_path)
load_model_safe(G_B2A, G_B2A_path)

# ============================================================
# 🔹 Image transforms
# ============================================================
transform_in = transforms.Compose([
    transforms.Resize((256,256)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5))
])
def postprocess_tensor(tensor):
    t = tensor.squeeze(0).cpu().detach()
    t = (t * 0.5) + 0.5
    t = t.clamp(0,1)
    return transforms.ToPILImage()(t)

# ============================================================
# 🔹 Auto-detect sketch vs photo
# ============================================================
def detect_sketch_or_photo(pil_img):
    img = np.array(pil_img.convert('RGB'))
    hsv = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
    sat_mean = hsv[:,:,1].mean() / 255.0
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    edges = cv2.Canny(gray, 100, 200)
    edge_density = edges.mean() / 255.0
    if sat_mean < 0.12 and edge_density > 0.02: return 'sketch'
    if sat_mean < 0.12: return 'sketch'
    return 'photo'

# ============================================================
# 🔹 Flask App with better frontend
# ============================================================
app = Flask(__name__)
CORS(app)

HTML_PAGE = '''
<!DOCTYPE html>
<html lang="en">
<head>
<meta charset="UTF-8">
<title>CycleGAN Face ↔ Sketch Converter</title>
<style>
body {
  background: #111827;
  color: #fff;
  font-family: 'Segoe UI', sans-serif;
  text-align: center;
  margin: 0; padding: 0;
}
h2 { margin-top: 30px; color: #60a5fa; }
.container {
  margin-top: 40px;
}
input[type=file] {
  background: #1f2937;
  border: 1px solid #374151;
  color: #fff;
  padding: 10px;
  border-radius: 8px;
}
button {
  background: #3b82f6;
  color: #fff;
  padding: 10px 20px;
  border: none;
  border-radius: 8px;
  cursor: pointer;
  margin-top: 15px;
}
button:hover { background: #2563eb; }
.preview {
  display: flex;
  justify-content: center;
  gap: 20px;
  margin-top: 30px;
  flex-wrap: wrap;
}
.preview img {
  width: 256px;
  height: 256px;
  object-fit: cover;
  border-radius: 10px;
  box-shadow: 0 0 15px rgba(255,255,255,0.1);
}
</style>
</head>
<body>
  <h2>🌀 CycleGAN Face ↔ Sketch Converter</h2>
  <div class="container">
    <input type="file" id="imageInput" accept="image/*">
    <br>
    <button onclick="uploadImage()">Convert</button>
    <div class="preview" id="preview"></div>
  </div>
<script>
async function uploadImage(){
  const fileInput = document.getElementById('imageInput');
  if (!fileInput.files.length) return alert('Please select an image.');
  const formData = new FormData();
  formData.append('image', fileInput.files[0]);
  const previewDiv = document.getElementById('preview');
  previewDiv.innerHTML = '<p>Processing...</p>';
  const res = await fetch('/convert', {method: 'POST', body: formData});
  const blob = await res.blob();
  const convertedUrl = URL.createObjectURL(blob);
  const originalUrl = URL.createObjectURL(fileInput.files[0]);
  previewDiv.innerHTML = `
    <div><h4>Original</h4><img src="${originalUrl}"></div>
    <div><h4>Converted</h4><img src="${convertedUrl}"></div>`;
}
</script>
</body>
</html>
'''

@app.route('/')
def home():
    return render_template_string(HTML_PAGE)

@app.route('/convert', methods=['POST'])
def convert():
    if 'image' not in request.files:
        return jsonify({'error':'No image uploaded'}), 400
    img_file = request.files['image']
    img = Image.open(img_file).convert('RGB')
    img_type = detect_sketch_or_photo(img)
    input_tensor = transform_in(img).unsqueeze(0).to(device)
    with torch.no_grad():
        output_tensor = G_B2A(input_tensor) if img_type == 'sketch' else G_A2B(input_tensor)
    output_img = postprocess_tensor(output_tensor)
    buf = io.BytesIO()
    output_img.save(buf, format='JPEG')
    buf.seek(0)
    return buf.getvalue(), 200, {'Content-Type': 'image/jpeg'}

# ============================================================
# 🔹 Ngrok + Run
# ============================================================
!ngrok config add-authtoken 32JOlU24yIl2AfXBgvXLN67t25F_5NwouvyuY2B6BR2HocywU

public_url = ngrok.connect(5000).public_url
print(f"🔥 Public URL: {public_url}")

app.run(port=5000)


✅ Google Drive already mounted.
Using device: cuda
✅ Loaded checkpoint: /content/drive/MyDrive/cyclegan_checkpoints/G_A2B_epoch30.pth
✅ Loaded checkpoint: /content/drive/MyDrive/cyclegan_checkpoints/G_B2A_epoch30.pth
Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
🔥 Public URL: https://c030d9140c10.ngrok-free.app
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [03/Nov/2025 18:13:07] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [03/Nov/2025 18:13:08] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [03/Nov/2025 18:14:01] "POST /convert HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [03/Nov/2025 18:16:32] "POST /convert HTTP/1.1" 200 -
